# Introduçãos as CNNs no Keras e ilustraçãos resultados obtidos para cada camada da DNN

Como funções de ativação, serão utilizados a função ReLU e a Sigmóide. 

**Sigmóide:**
A função sigmoide tem a forma de um "S". Ela é definida pela seguinte equação:

$$S(x) = \frac{1}{1+e^{-x}}$$

![](https://sebastianraschka.com/images/faq/logisticregr-neuralnet/sigmoid.png)

**ReLU:**
A função ReLU também chamda de fnção rampa é definida pela seguinte equação:

$$f(x) = max(0,x)$$

Simplesmente, ela é ativada caso o valor de entrada é maior que zero não alterando o mesmo, caso contrário o valor de entrada é multiplicado por zero na saída.

![](http://csci431.artifice.cc/images/relu.png)

## Importação dos Módulos

In [1]:
import numpy as np
import os
os.environ['THEANO_FLAGS'] = "device=gpu0,floatX=float32" 
from keras.models import Model, Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D, Input

Using Theano backend.
Using gpu device 0: Tesla K40c (CNMeM is disabled, cuDNN Version is too old. Update to v5, was 2000.)


## Definição da CNN

Uma CNN pode ser construída de dois modos no Keras. Ela pode ser implementada usando a API, ou seja importa os modelos e introduzindo suas entradas e saídas, ou usando o modelo Sequential em que as camadas são postas diretamente uma abaixo da outra sem a necessidade de explicitamente indicar suas entradas e saídas. Veja os exemplos a seguir:


In [2]:
#### Alterar #####
n_filters = 3
filter_size = 2 # kerrnel filter_sizexfilter_size
f_bias = 0.1 # valor de bias
##################

# Tamanho da entrada
img_rows = 5
img_cols = 5

# Pesos para as camadas de convolução e densa
W_conv = np.ones(shape=(filter_size,1))
aux = np.ones(filter_size*n_filters)
W_conv+aux # Broadcasting para repetir o kernel no número dado por n_filters
W_conv  = (W_conv+aux).reshape(n_filters,1,filter_size,filter_size)

#W_conv = np.ones(shape=(2,2)).reshape(1,1,2,2)
bias_conv = np.ones(n_filters)*f_bias
bias_dense = np.ones(1)*f_bias
n = filter_size*filter_size*n_filters
W_dense = np.ones(shape=(n)).reshape(n,1)
input_shape = (1, img_rows, img_cols)
# para desabilitar o bias basta inserir como parâmetro na camada bias = False e apagar ele no weights
    
def get_cnn_model_API():
    
    inputs = Input(input_shape, name = 'input')
    conv1 = Convolution2D(n_filters, filter_size, filter_size, 
                          weights = [W_conv,bias_conv],
                          name = 'conv1', border_mode='valid')(inputs) 
    actv1 = Activation('relu', name = 'actv1')(conv1)
    pool1 = MaxPooling2D(pool_size=(2,2), name = 'max_pool1')(actv1) 
    flat = Flatten(name = 'flat')(pool1)
    dense1 = Dense(1, weights = [W_dense,bias_dense], name = 'dense1')(flat)
    out = Activation('sigmoid', name = 'out_sig')(dense1)

    model = Model(input=inputs, output=out)
    
    return model

def get_cnn_model_Sequential():
       
    model = Sequential()
    
    model.add(Convolution2D(n_filters, filter_size, filter_size, 
                          weights = [W_conv,bias_conv],
                          name = 'conv1', border_mode='valid',
                          input_shape=input_shape))
    model.add(Activation('relu', name = 'actv1'))
    model.add(MaxPooling2D(pool_size=(2, 2), name = 'max_pool1'))
    model.add(Flatten(name = 'flat'))
    model.add(Dense(1, weights = [W_dense,bias_dense], name = 'dense1'))
    model.add(Activation('sigmoid', name = 'out_sig'))
    
    return model

# Resultados da CNN para uma dada entrada

## Exemplo com usando a API do Keras

In [8]:
np.set_printoptions(precision=3)

# Leitura do modelo
#model = get_cnn_model_API()
model = get_cnn_model_Sequential()

# Entrada
X = np.array([[[[1,0,0,0,0],
                [0,1,0,0,0],
                [0,0,-1,0,0],
                [0,0,0,-1,0],
                [0,0,0,0,1]]]])



print('-'*30)
print "Número de camadas:", len(model.layers)
print('-'*30)

# Resultados para cada camada
i = 1
for layer in model.layers:
    intermediate_layer_model = Model(input=model.input,output=layer.output)
    intermediate_output = intermediate_layer_model.predict(X)
    print('-'*80)
    print "Saída da camada", i, ":", layer.name
    print('-'*80)
    print intermediate_output
    i+=1

# Sumário contendo informações de cada camada da CNN, como nome, tipo de camada número de parâmetros e conexões
print model.summary()

------------------------------
Número de camadas: 6
------------------------------
--------------------------------------------------------------------------------
Saída da camada 1 : conv1
--------------------------------------------------------------------------------
[[[[ 4.1  2.1  0.1  0.1]
   [ 2.1  0.1 -1.9  0.1]
   [ 0.1 -1.9 -3.9 -1.9]
   [ 0.1  0.1 -1.9  0.1]]

  [[ 4.1  2.1  0.1  0.1]
   [ 2.1  0.1 -1.9  0.1]
   [ 0.1 -1.9 -3.9 -1.9]
   [ 0.1  0.1 -1.9  0.1]]

  [[ 4.1  2.1  0.1  0.1]
   [ 2.1  0.1 -1.9  0.1]
   [ 0.1 -1.9 -3.9 -1.9]
   [ 0.1  0.1 -1.9  0.1]]]]
--------------------------------------------------------------------------------
Saída da camada 2 : actv1
--------------------------------------------------------------------------------
[[[[ 4.1  2.1  0.1  0.1]
   [ 2.1  0.1  0.   0.1]
   [ 0.1  0.   0.   0. ]
   [ 0.1  0.1  0.   0.1]]

  [[ 4.1  2.1  0.1  0.1]
   [ 2.1  0.1  0.   0.1]
   [ 0.1  0.   0.   0. ]
   [ 0.1  0.1  0.   0.1]]

  [[ 4.1  2.1  0.1  0.1]
   [ 

## Sugestões de atividades

1. Executar o experimento nos dois modos, API e Sequential.
2. Mudar paramêtros das camadas convolucionais e analisar saídas. (filter_size e n_filters)
3. Verificar relação entre número de parâmetros e filtros de convolução.
4. Adicionar e retirar bias e comparar resultados do número de parâmetros e das saídas. (f_bias)


## Referências

- [Keras Documentation](https://keras.io/)


## Aprendizados com este notebook
